<a href="https://colab.research.google.com/github/RamsesT/MVA_DeepLearningCourse/blob/master/DQN_project_MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

np.random.seed(1)


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

the function **act** is used to decide what action the agent should do. If *train* is set to **True** it will use epsilon to explore its evironement. *epsilon* is the probability that the agent will follow the action based on the best policy learned so far or if it will act randomly. This exploration is important for the agent to learn new behaviours and not just focus on a small set of strategie that it has tested. 
By setting espilon to a large value we force the agent to explore. If epsilon is small it will use what it learned.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=31 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

 ```position``` is an array which contains the information on the position of the rat(set to 1), the places it can go (set to 0) and where he can not (the boundaries, set to -1).

```board``` is an array with the poison and food positions (-1 for poison, 0.5 for food). The probability to found poison or food on a given position follows a binomial law with expectation equal to *self.temperature*
 
 Both arrays have the same shape.
 
 After each action, the rate eat what it founds in its new position. At the corresponding new position, we set the value on the board to 0 to show that there is nothing left here.
  


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            action = agent.act(state, train=False) # no need to use the "train" feature since its a random agent

            prev_state = state
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/13.0. Average score (-3.0)
Win/lose count 9.0/18.0. Average score (-6.0)
Win/lose count 9.5/11.0. Average score (-4.5)
Win/lose count 13.0/20.0. Average score (-5.125)
Win/lose count 8.0/17.0. Average score (-5.9)
Win/lose count 8.0/11.0. Average score (-5.416666666666667)
Win/lose count 7.0/12.0. Average score (-5.357142857142857)
Win/lose count 9.5/13.0. Average score (-5.125)
Win/lose count 11.5/20.0. Average score (-5.5)
Win/lose count 8.5/11.0. Average score (-5.2)
Win/lose count 8.5/12.0. Average score (-5.045454545454546)
Win/lose count 10.5/15.0. Average score (-5.0)
Win/lose count 12.0/4.0. Average score (-4.0)
Win/lose count 9.0/12.0. Average score (-3.9285714285714284)
Win/lose count 12.5/17.0. Average score (-3.966666666666667)
Win/lose count 6.5/15.0. Average score (-4.25)
Win/lose count 9.5/15.0. Average score (-4.323529411764706)
Win/lose count 11.5/18.0. Average score (-4.444444444444445)
Win/lose count 11.5/18.0. Average score (-4.552631578947368)
W

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**proof**

From the course we have : 
\begin{align}
Q^{\pi}(s,a) = E[\sum\limits_{k=0}^\infty \gamma^k r_{t+k}|s_t=s,a_t=a ] \\
Q^{\pi}(s,a) = E[r(s,a) + \sum\limits_{k=1}^\infty \gamma^k r_{t+k}|s_t=s,a_t=a] \\
Q^{\pi}(s,a) = r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
Q^{\pi}(s,a)  =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
Q^{\pi}(s,a) = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

Then we maximise this quantity: 

\begin{align}
Q^{\ast}(s,a) =  \max\limits_{\pi} Q^{\pi}(s,a) \\
Q^{\ast}(s,a) = \max\limits_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
Q^{\ast}(s,a) = r(s,a)+ \gamma\max\limits_{\pi} E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]\\
\end{align}

We have to show that we can switch the $\max$ and $\sum$ operations :


\begin{align}
 \sum_{s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') = \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\pi^\ast (s',a')}(s',a') \\
  \sum_{s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') \leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align}

With $\pi^\ast$ the best policy.

The other inequality is tivial (the maximum of a sum $\leq$ sum of the maximum).

We then have : 


\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}


The proposed objective functions forces the network to found a solution that verifies the Bellman equation.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[1:]

    def random_access(self):
        return self.memory[np.random.randint(0,len(self.memory),1)[0]]
         

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        for i in range(self.batch_size):
            ######## FILL IN
            s_i, n_s_i, a_i, r_i, game_over_i  = self.memory.random_access()
            target_q[i] =self.model.predict(s_i.reshape([1,s_i.shape[0],s_i.shape[1],s_i.shape[2]]))[0]
            input_states[i] = s_i
            if game_over_i:
                ######## FILL IN
                target_q[i,a_] = r_i
            else:
                ######## FILL IN
                #target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape((1,s_.shape[0],s_.shape[1],s_.shape[2]))[0]))
                target_q[i,a_] = r_i  + self.discount*max(self.model.predict(np.expand_dims(s_i, axis=0))[0])
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l
    

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        #model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Reshape((50,), input_shape=(5,5,self.n_state)))
        model.add(Dense(30,activation ='relu')) 
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/010 | Loss 0.0006 | Win/lose count 4.5/1.0 (3.5)
Epoch 001/010 | Loss 0.0198 | Win/lose count 2.5/7.0 (-4.5)
Epoch 002/010 | Loss 0.0038 | Win/lose count 0.5/3.0 (-2.5)
Epoch 003/010 | Loss 0.0090 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/010 | Loss 0.0484 | Win/lose count 4.0/6.0 (-2.0)
Epoch 005/010 | Loss 0.0015 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/010 | Loss 0.0055 | Win/lose count 4.0/7.0 (-3.0)
Epoch 007/010 | Loss 0.0220 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/010 | Loss 0.0741 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/010 | Loss 0.0126 | Win/lose count 6.0/8.0 (-2.0)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(5,(3,3), padding='valid'))
        model.add(Conv2D(25,(3,3), padding='valid'))
        model.add(Flatten())
        model.add(Dense(4)) 
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')

Epoch 000/010 | Loss 0.0381 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/010 | Loss 0.0068 | Win/lose count 2.5/7.0 (-4.5)
Epoch 002/010 | Loss 0.0185 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/010 | Loss 0.0047 | Win/lose count 9.5/8.0 (1.5)
Epoch 004/010 | Loss 0.0111 | Win/lose count 8.5/9.0 (-0.5)
Epoch 005/010 | Loss 0.0146 | Win/lose count 4.0/10.0 (-6.0)
Epoch 006/010 | Loss 0.0026 | Win/lose count 5.0/8.0 (-3.0)
Epoch 007/010 | Loss 0.0071 | Win/lose count 7.5/11.0 (-3.5)
Epoch 008/010 | Loss 0.0102 | Win/lose count 8.0/11.0 (-3.0)
Epoch 009/010 | Loss 0.0230 | Win/lose count 4.5/4.0 (0.5)


In [0]:

HTML(display_videos('cnn_train50.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 2.5/1.0. Average score (0.5)
Win/lose count 0/1.0. Average score (0.0)
Win/lose count 0/1.0. Average score (-0.25)
Win/lose count 1.0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0.5/0. Average score (0.07142857142857142)
Win/lose count 1.5/1.0. Average score (0.125)
Win/lose count 0.5/2.0. Average score (-0.05555555555555555)
Win/lose count 0/1.0. Average score (-0.15)
Win/lose count 0/0. Average score (-0.13636363636363635)
Win/lose count 2.0/2.0. Average score (-0.125)
Win/lose count 1.0/1.0. Average score (-0.11538461538461539)
Win/lose count 0/2.0. Average score (-0.25)
Win/lose count 2.0/1.0. Average score (-0.16666666666666666)
Win/lose count 1.5/0. Average score (-0.0625)
Win/lose count 2.0/0. Average score (0.058823529411764705)
Win/lose count 0/0. Average score (0.05555555555555555)
Win/lose count 0.5/0. Average score (0.07894736842105263)
Win/lose count 0/0. Average s

In [0]:
HTML(display_videos('cnn_test9.mp4'))

In [0]:
HTML(display_videos('fc_test9.mp4'))

Based on the tests I made, CNN with 2 convolutionnal layers seems to perform best on this task.

As the temperature increases, both models perform better, but the CNN score growth is faster than the one of the fully connected layer.
One issue of the models trains is that it will stop increasing in performance because it would rather go on empty spots than try to eat something that might be poisoned and got a poor score. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:

## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)


def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    #decay_parameter_epsilon in order to use the decreasing $\epsilon$-greedy exploration
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) 
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [0]:
# Training
epochs_train=81
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train,decay_parameter_epsilon=0.1, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore80.mp4'))

Epoch 000/081 | Loss 0.0180 | Win/lose count 8.5/22.90000000000003 (-14.40000000000003)
Epoch 001/081 | Loss 0.0292 | Win/lose count 8.0/21.400000000000038 (-13.400000000000038)
Epoch 002/081 | Loss 0.0234 | Win/lose count 6.0/26.400000000000084 (-20.400000000000084)
Epoch 003/081 | Loss 0.0149 | Win/lose count 10.0/28.40000000000004 (-18.40000000000004)
Epoch 004/081 | Loss 0.0168 | Win/lose count 11.5/38.20000000000012 (-26.700000000000117)
Epoch 005/081 | Loss 0.0320 | Win/lose count 8.0/26.90000000000009 (-18.90000000000009)
Epoch 006/081 | Loss 0.0318 | Win/lose count 9.0/28.800000000000097 (-19.800000000000097)
Epoch 007/081 | Loss 0.0101 | Win/lose count 13.5/34.60000000000009 (-21.100000000000087)
Epoch 008/081 | Loss 0.0175 | Win/lose count 6.0/22.300000000000054 (-16.300000000000054)
Epoch 009/081 | Loss 0.0058 | Win/lose count 7.0/26.90000000000006 (-19.90000000000006)
Epoch 010/081 | Loss 0.0250 | Win/lose count 7.5/29.000000000000085 (-21.500000000000085)
Epoch 011/081 | L

In [0]:
 # Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore30.mp4'))

Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 0.5/0. Average score (-0.16666666666666666)
Win/lose count 1.5/3.0. Average score (-0.5)
Win/lose count 0/1.0. Average score (-0.6)
Win/lose count 1.5/2.0. Average score (-0.5833333333333334)
Win/lose count 0.5/1.0. Average score (-0.5714285714285714)
Win/lose count 0.5/0. Average score (-0.4375)
Win/lose count 1.5/0. Average score (-0.2222222222222222)
Win/lose count 0/0. Average score (-0.2)
Win/lose count 1.0/0. Average score (-0.09090909090909091)
Win/lose count 1.5/0. Average score (0.041666666666666664)
Win/lose count 0.5/2.0. Average score (-0.07692307692307693)
Win/lose count 0/0. Average score (-0.07142857142857142)
Win/lose count 0.5/3.0. Average score (-0.23333333333333334)
Win/lose count 0.5/2.0. Average score (-0.3125)
Win/lose count 4.0/2.0. Average score (-0.17647058823529413)
Win/lose count 2.0/2.0. Average score (-0.16666666666666666)
Win/lose count 1.5/2.0. Average

The model seems to work better during the training set (based on the video). On the test part I can't see any improvement.
I noticed that the number of win/loss game is always negative. I tried to investigate where this could came from in the code but I didn't found where is my bug.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***